In [ ]:
USE ROLE ACCOUNTADMIN;
USE DATABASE IPO_RESEARCH_DB;

CREATE OR REPLACE NETWORK RULE stock_api_network_rule MODE = EGRESS TYPE = HOST_PORT VALUE_LIST = (
    'finnhub.io',
    'api.finnhub.io'
) COMMENT = 'Allow access to Finnhub stock API';

-- Create external access integration                                                                                                                                            
CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION stock_api_integration 
ALLOWED_NETWORK_RULES = (stock_api_network_rule) 
ENABLED = TRUE 
COMMENT = 'Integration for stock price API access';

GRANT USAGE ON INTEGRATION stock_api_integration TO ROLE SYSADMIN;


In [ ]:
-- Store your API key as a secret                                                                                                                                                
CREATE OR REPLACE SECRET finnhub_api_key TYPE = GENERIC_STRING SECRET_STRING = '<INSERT YOUR API KEY HERE>';

-- Grant usage                                                                                                                                                                   
GRANT USAGE ON SECRET finnhub_api_key TO ROLE SYSADMIN;

ALTER EXTERNAL ACCESS INTEGRATION stock_api_integration SET ALLOWED_AUTHENTICATION_SECRETS = (finnhub_api_key);


In [ ]:
USE ROLE SYSADMIN;

CREATE OR REPLACE PROCEDURE get_stock_price(ticker VARCHAR) 
RETURNS OBJECT 
LANGUAGE PYTHON RUNTIME_VERSION = '3.10' 
PACKAGES = ('requests', 'snowflake-snowpark-python') 
HANDLER = 'get_stock_price' 
EXTERNAL_ACCESS_INTEGRATIONS = (stock_api_integration) 
SECRETS = ('api_key' = finnhub_api_key) 
AS 
$$ 
import requests 
import _snowflake 

def get_stock_price(session, ticker): 
    """Fetch real-time stock price from Finnhub API""" 
    
    # Get API key from secret
    api_key = _snowflake.get_generic_secret_string('api_key') 
    
    # Normalize ticker
    ticker = ticker.upper().strip() 
    
    # Fetch quote data
    url = f"https://finnhub.io/api/v1/quote?symbol={ticker}&token={api_key}" 

    try: 
        response = requests.get(url, timeout = 10) 
        response.raise_for_status() 
        data = response.json() 
        
        # Check if valid data returned                                                     
        if data.get('c') == 0 and data.get('pc') == 0: 
            return { 
                'error': f'Invalid ticker symbol: {ticker}',
                'ticker': ticker 
            } 
                
        # Return structured data           
        return { 
            'ticker': ticker,
            'current_price': data.get('c'),
            'change': data.get('d'),
            'percent_change': data.get('dp'),
            'high': data.get('h'),
            'low': data.get('l'),
            'open': data.get('o'),
            'previous_close': data.get('pc'),
            'timestamp': data.get('t') 
        }
    except requests.exceptions.RequestException as e: 
            return { 
                'error': f'API request failed: {str(e)}',
                'ticker': ticker 
            }
    except Exception as e: 
        return { 
            'error': f'Unexpected error: {str(e)}',
            'ticker': ticker 
        } 
$$;

In [ ]:
CREATE OR REPLACE PROCEDURE get_stock_fundamentals(ticker VARCHAR) 
RETURNS OBJECT 
LANGUAGE PYTHON RUNTIME_VERSION = '3.10' 
PACKAGES = ('requests', 'snowflake-snowpark-python') 
HANDLER = 'get_stock_fundamentals' 
EXTERNAL_ACCESS_INTEGRATIONS = (stock_api_integration) 
SECRETS = ('api_key' = finnhub_api_key) 
AS 
$$ 
import requests 
import _snowflake 

def get_stock_fundamentals(session, ticker): 
    """Fetch basic financials from Finnhub API""" 
    
    # Get API key from secret
    api_key = _snowflake.get_generic_secret_string('api_key') 
    
    # Normalize ticker
    ticker = ticker.upper().strip() 
    
    # Fetch basic financials data
    url = f"https://finnhub.io/api/v1/stock/metric?symbol={ticker}&metric=all&token={api_key}" 

    try: 
        response = requests.get(url, timeout=10) 
        response.raise_for_status() 
        data = response.json() 
        
        # Check if valid data returned
        if not data or 'metric' not in data:
            return { 
                'error': f'No fundamental data available for ticker: {ticker}',
                'ticker': ticker 
            } 
        
        metrics = data.get('metric', {})
        series = data.get('series', {})
        
        # Return structured fundamental data
        return { 
            'ticker': ticker,
            'market_cap': metrics.get('marketCapitalization'),
            'pe_ratio': metrics.get('peBasicExclExtraTTM'),
            'pb_ratio': metrics.get('pbAnnual'),
            'ps_ratio': metrics.get('psAnnual'),
            'peg_ratio': metrics.get('pegRatioTTM'),
            'enterprise_value': metrics.get('enterpriseValue'),
            'ev_ebitda': metrics.get('evEbitdaTTM'),
            'revenue_ttm': metrics.get('revenueTTM'),
            'gross_margin_ttm': metrics.get('grossMarginTTM'),
            'operating_margin_ttm': metrics.get('operatingMarginTTM'),
            'net_margin_ttm': metrics.get('netProfitMarginTTM'),
            'roe': metrics.get('roeTTM'),
            'roa': metrics.get('roaTTM'),
            'roi': metrics.get('roiTTM'),
            'debt_to_equity': metrics.get('totalDebt/totalEquityAnnual'),
            'current_ratio': metrics.get('currentRatioAnnual'),
            'quick_ratio': metrics.get('quickRatioAnnual'),
            'dividend_yield': metrics.get('dividendYieldIndicatedAnnual'),
            'beta': metrics.get('beta'),
            '52_week_high': metrics.get('52WeekHigh'),
            '52_week_low': metrics.get('52WeekLow'),
            '52_week_high_date': metrics.get('52WeekHighDate'),
            '52_week_low_date': metrics.get('52WeekLowDate'),
            'shares_outstanding': metrics.get('sharesOutstanding'),
            'float_shares': metrics.get('floatShares'),
            'annual_revenue_series': series.get('annual', {}).get('revenue', []),
            'annual_eps_series': series.get('annual', {}).get('eps', [])
        }
        
    except requests.exceptions.RequestException as e: 
        return { 
            'error': f'API request failed: {str(e)}',
            'ticker': ticker 
        }
    except Exception as e: 
        return { 
            'error': f'Unexpected error: {str(e)}',
            'ticker': ticker 
        } 
$$;

In [ ]:
CALL get_stock_fundamentals('SNOW');      